# Processamento da Fonte de Dados

Este notebook realiza o processo de ETL (extract, transform, load) para as fontes de dados disponibilizadas pelo [Mapa das Organizações da Sociedade Civil](https://mapaosc.ipea.gov.br/base-dados).

## Configurando o Ambiente

Para manutenção da simplicidade, os códigos necessários para realizar o processamento dos dados estão segmentados em módulos. Este caderno tem o propósito apenas de orquestrar e exibir os resultados dos processamentos realizados por tais módulos.

Para isto, é necessário reconfigurar o ambiente do notebook, para que os caminhos dos módulos sejam resolvidos corretamente, como em uma aplicação convencional Python.


In [ ]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Added to Python path: {project_root}")
print(f"Current working directory: {os.getcwd()}")

## Etapas do Processamento

Este caderno executa as seguintes etapas para produzir uma base de dados tratada. Assume-se que as fontes de dados brutas estão disponíveis no diretório `datasets` deste repositório.

### 1. Conversão de Encoding

Os datasets disponibilizados encontram-se em diversos formatos, além de contarem com "encodings" inadequados para a leitura e análise adequada através de algoritmos.

Para cada dataset original, esta etapa gera fontes contendo encoding adequado (em utf8) para a realização de etapas posteriores.


In [ ]:
from data.processing.data_parser import to_utf8

ONGS_DATASET = "../datasets/osc_2025_2.csv"
PROJECTS_DATASET = "../datasets/projetos.csv"

parsed_ongs_source = to_utf8(ONGS_DATASET,
                             separator=";",
                             input_encoding="latin1")
parsed_projects_source = to_utf8(PROJECTS_DATASET)

if parsed_ongs_source is None:
    raise FileNotFoundError(
        f"File {ONGS_DATASET} not found or could not be converted to UTF-8.")

if parsed_projects_source is None:
    raise FileNotFoundError(
        f"File {PROJECTS_DATASET} not found or could not be converted to UTF-8.")

### 2. Tratamento de Organizações

A segunda fase lida com as informações disponibilizadas na base `osc_2025_2.csv`, que contém informações sobre as OSCs catalogadas na fonte mencionada.

#### 2.1. Tratameno de Identificadores

Para garantir que possíveis referências cruzadas sejam executadas corretamente, o CNPJ, originalmente vindo somente como dígitos (e sem zeros à esquerda), é formatado
para o formato convencional `XX.XXX.XXX/XXXX-XX`:

| Cnpj (Fonte)   | Cnpj (Formatado)   |
| -------------- | ------------------ |
| 00000000000    | 00.000.000/0000-00 |
| 11111111111111 | ...                |

#### 2.2. Tratamento de Áreas de Atuação

O dataset original armazena os códigos de área de atuação de cada OSC de maneira tabular, conforme o exemplo abaixo:

| Cnpj               | ... | Area_x | Area_y | Area_z |
| ------------------ | --- | ------ | ------ | ------ |
| 00.000.000/0000-00 | ... | 0      | 1      | 0      |
| 11.111.111/1111-11 | ... | 1      | 0      | 1      |

Onde valores `1` indicam que a OSC faz parte da área de atuação correspondente à coluna.

Para melhorar a legibilidade do dataset final, esta estrutura é trauzida e concatenada em um único texto descritivo, conforme o exemplo abaixo:

| Cnpj               | ... | Areas de Atuação |
| ------------------ | --- | ---------------- |
| 00.000.000/0000-00 | ... | Area y           |
| 11.111.111/1111-11 | ... | Area x, Area z   |


In [ ]:
from data.processing.data_parser import write_dataset
from data.processing.osc import osc_dataset
from pandas import read_csv

osc_source = read_csv(parsed_ongs_source, sep=";", dtype=str)
osc_df = osc_dataset(osc_source)
osc_path = write_dataset("osc", osc_df)

if (osc_path is None) or (not os.path.isfile(osc_path)):
    raise FileNotFoundError(
        f"File {osc_path} not found or could not be created.")

In [ ]:
osc_df.head(20)

### 3. Tratamento de Projetos

Projetos representam as ações sociais realizadas pelas OSCs.
Para garantir sanitização adequada de informações,
o dataset final gerado pelo ETL aplica os seguintes critérios de filtragem:

- Somente projetos que forneçam data de início;
- Somente projetos que contenham ao menos uma das seguintes informações:
  - O total de beneficiados;
  - O valor captado;
  - O valor total;
- Somente projetos do Distrito Federal

> Além disso, duplicatas serão removidas do dataset final. Para este dataset, considera-se uma duplicata projetos que contenham o mesmo valor para a coluna "Id Projeto".

#### 3.1. Filtrando Projetos por Região

Internamente, o método `projects_dataset` aplica um filtro para retornar somente projetos cuja região esteja no Distrito Federal. Para isso, o método `projects.by_region` é utilizado para realizar o cruzamento de referência entre as colunas "CNPJ OSC", do dataset de projetos, e "CNPJ", do dataset de OSCs.


In [ ]:
from data.processing.data_parser import write_dataset
from data.processing.projects import projects_dataset
from pandas import read_csv

projects_source = read_csv(parsed_projects_source, sep=";", dtype=str)
projects_df = projects_dataset(projects_source, osc_df)
projects_path = write_dataset("projects", projects_df)

if (projects_path is None) or (not os.path.isfile(projects_path)):
    raise FileNotFoundError(
        f"File {projects_path} not found or could not be created.")

In [ ]:
projects_df.head(20)

In [ ]:
from model.decision_tree_model import DecisionTreeModel
from model.naive_bayes_model import NaiveBayesModel
from model.svm_model import SVMModel
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Carregar o dataset tratado
dataset_path = projects_path
data = pd.read_csv(dataset_path, sep=";", dtype=str)
data.fillna('', inplace=True)

# Codificar variáveis categóricas
label_encoders = {}
for column in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Separar features e target
X = data.drop('Status', axis=1)
y = data['Status']

# Treinar e avaliar modelos
models = {
    'Decision Tree': DecisionTreeModel(),
    'Naive Bayes': NaiveBayesModel(),
    'SVM': SVMModel()
}

for model_name, model in models.items():
    print(f"\nTraining and evaluating {model_name}...")
    
    # Atribuir os dados ao modelo
    model.X = X
    model.y = y
    
    model.train()
    model.evaluate() 